In [5]:
# 필요 패키지 임포드
import pandas
import numpy#
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD, PCA, NMF
import matplotlib.pyplot as plt
import seaborn
import scipy

%matplotlib inline

#
# 1. DATA LOAD
# 
# Kaggle에서 제공하는 [BNP Paribas Cardif Claims Management](https://www.kaggle.com/c/bnp-paribas-cardif-claims-management/data)데이터를 활용
# BNP Paribas Cardif 청구관리 데이터 셋으로 고객의 클레임 여부를 판단하는 데이터셋의 Subset
DATA_PATH = "c:\\Users\\WAI\\OneDrive\\03.WORKSPACE\\00.DATA\\"
df = pandas.read_csv(DATA_PATH + "BookExample.csv")
print(df.shape) # (20000, 33)
target_df = df[["target"]]
df.set_index(["ID","target"], inplace=True)

(20000, 33)


In [6]:
# 범주화 및 인코딩 하기전의 데이터 셋
n_obj_cols = []
for col, type in zip(df.dtypes.index, df.dtypes.values):
    if type != ("object"):
        n_obj_cols.append(col)

n_obj_df = df[n_obj_cols]
print(n_obj_df.shape)

# PCA를 하기전에 Scale 조정을 통해 각 특성의 분산량을 동일 하게 수정
x = n_obj_df.values
x = StandardScaler().fit_transform(x)
features = n_obj_cols
print(pandas.DataFrame(x, columns = features).head())
#print(pandas.DataFrame(x, columns = features).describe())

(20000, 26)
         v1        v2        v4        v5        v6        v7        v8  \
0  0.358438  1.039820  0.704778  0.627939  0.949960  1.352400 -0.389224   
1 -0.811045 -0.982284 -1.056727  0.907032 -1.075637 -1.080416  0.640466   
2  0.015350  0.248030  0.924883  0.061637  2.025628  1.928022 -0.386208   
3 -0.112883  0.941879  0.841755  1.439884  0.559084  0.441618 -0.317687   
4 -0.811045 -0.982284 -1.056727 -1.103328 -1.075637 -1.080416 -0.395047   

         v9       v10       v11  ...       v18       v19       v20       v21  \
0  1.037773 -0.984941  0.996999  ... -0.591831  0.167826  0.992835  0.628456   
1 -1.091080 -0.405231 -1.143926  ... -0.760831 -0.933717 -1.141823 -0.206725   
2  1.605468 -0.796927  0.778399  ... -0.373581  0.133608  0.889102 -1.516758   
3  0.817547  3.339377  0.985714  ...  1.177649  0.780048  0.937077  0.443957   
4 -1.091080 -0.593245 -1.143926  ... -0.760831 -0.933717 -1.141823 -0.507503   

        v23       v25       v26       v27       v28     

In [15]:
# SVD(Single Value Decomposition) - 특이값 분해
# scikit learn, numpy, scipy 에서 각기의 방식으로 API를 제공합니다.
# scikit learn의 truncatedSVD를 사용해보록 하겠습니다.

# 희소 행렬 생성
features_sparse = scipy.sparse.csr_matrix(x)

# PCA와 동일한 components로 구성하여 Explained Variance를 비교하도록 하겠습니다. 
tsvd = TruncatedSVD(n_components=5)

tsvdComponents = tsvd.fit_transform(x)
tsvd_df = pandas.DataFrame(data = tsvdComponents, columns = ['pcaComp1', 'pcaComp2', 'pcaComp3', 'pcaComp4', 'pcaComp5'])
print(tsvd_df.head())

# 결과 출력
print("원본 특성 개수:", features_sparse.shape[1]) 
print("줄어든 특성 개수:", tsvdComponents.shape[1])
print(tsvd.explained_variance_ratio_)
print("원본 데이터 설명률 : {}%".format(round(sum(tsvd.explained_variance_ratio_),5)*100))

# PCA와 결과가 동일한 것을 확인할 수 있습니다. 

   pcaComp1  pcaComp2  pcaComp3  pcaComp4  pcaComp5
0  2.984104 -0.816479 -1.753298 -0.201385 -1.135974
1 -3.794804 -0.620447  1.251827 -1.037672  0.010489
2  3.281121 -2.173977 -1.945966 -0.130781  1.109657
3  3.509760  4.156012 -0.358543  1.079690  1.904907
4 -4.505569 -1.130443 -0.059982 -0.126100  0.073080
원본 특성 개수: 26
줄어든 특성 개수: 5
[0.59187523 0.10849578 0.07850286 0.04393941 0.03710323]
원본 데이터 설명률 : 85.992%
